In [2]:
!pip install beautifulsoup4 lxml

    100% |████████████████████████████████| 92kB 786kB/s ta 0:00:01
    100% |████████████████████████████████| 5.6MB 351kB/s ta 0:00:011


In [1]:
import re
import json

import requests
from bs4 import BeautifulSoup

# get total page

In [2]:
URL = "https://www.1111.com.tw/job-bank/job-index.asp?si=1&ss=s&ks=java&page=1"

resp = requests.get(URL)

soup = BeautifulSoup(resp.text, 'lxml') # Beautifulsoup Object

In [3]:
page_info = soup.find("div", {"class":"pagedata"}).text
page_info

'第 1 / 50 頁，共 999 筆'

In [4]:
page = re.findall('第 [0-9]+ / ([0-9]+) 頁', page_info)
page

['50']

In [5]:
def get_total_page_cnt(url):
    #URL = "https://www.1111.com.tw/job-bank/job-index.asp?si=1&ss=s&ks=java&page=1"
    resp = requests.get(url)
    
    soup = BeautifulSoup(resp.text, 'lxml') # Beautifulsoup Objectbt
    page_info = soup.find("div", {"class":"pagedata"}).text
    page = int(re.findall('第 [0-9]+ / ([0-9]+) 頁', page_info)[0])
    
    return page

### Failed

resp = requests.get("https://www.1111.com.tw/job-bank/job-index.asp?si=1&ss=s&ks=java&page=2")

soup = BeautifulSoup(resp.text, 'lxml') # Beautifulsoup Object

web = soup.find('div', {'class':'jbInfoin'}).find('h3').find('a')['href']
resp = requests.get("https:" + web)
resp.text[30000:50000]

webs_info = soup.select('div.jbInfoin > h3 > a')
webs_info[:2]

# get all jobs links

In [6]:
def get_list_page(url):
    resp = requests.get(url)
    links = re.findall('class="jbInfoin">.+<a href=\"(.+)\" title=\".+\" target=\"_blank\">', resp.text)
    detail_page_links = ["https:" + link for link in links]
    return  detail_page_links

In [7]:
get_list_page("https://www.1111.com.tw/job-bank/job-index.asp?si=1&ss=s&ks=java&page=2")[:3]

['https://www.1111.com.tw/job/79839490/',
 'https://www.1111.com.tw/job/59308016/',
 'https://www.1111.com.tw/job/79958667/']

### [Failed] 一次爬multiple頁
URL_TEMPLATE = "https://www.1111.com.tw/job-bank/job-index.asp?si=1&ss=s&ks=java&page={}"

total_page_cnt = get_total_page_cnt(URL_TEMPLATE.format(1))

links = []

for pg in range(1, total_page_cnt):

    url = URL_TEMPLATE.format(pg)
    
    _links = [link for link in get_list_page(url)]
    
    links.append(_links)
    
return links

In [8]:
def get_all_page(url):
    URL_TEMPLATE = url + "&page={}"

    total_page_cnt = get_total_page_cnt(URL_TEMPLATE.format(1))
    
    links = []
    for pg in range(1, total_page_cnt):
        _url = URL_TEMPLATE.format(pg) #acutual website
        ### under error
        links.append([link for link in get_list_page(_url)])
    return links

In [9]:
all_page = get_all_page("https://www.1111.com.tw/job-bank/job-index.asp?si=1&ss=s&ks=java")

In [10]:
all_page[:2]

[['https://www.1111.com.tw/job/80197415/?agent=sticktop_71952995_80197415',
  'https://www.1111.com.tw/job/80057790/',
  'https://www.1111.com.tw/job/79828717/',
  'https://www.1111.com.tw/job/55576904/',
  'https://www.1111.com.tw/job/76821519/',
  'https://www.1111.com.tw/job/80231208/',
  'https://www.1111.com.tw/job/76812185/',
  'https://www.1111.com.tw/job/76812146/',
  'https://www.1111.com.tw/job/76812082/',
  'https://www.1111.com.tw/job/76812156/',
  'https://www.1111.com.tw/job/80108214/',
  'https://www.1111.com.tw/job/76812071/',
  'https://www.1111.com.tw/job/76812055/',
  'https://www.1111.com.tw/job/78573375/',
  'https://www.1111.com.tw/job/79932614/',
  'https://www.1111.com.tw/job/80120936/',
  'https://www.1111.com.tw/job/80078480/',
  'https://www.1111.com.tw/job/78356616/',
  'https://www.1111.com.tw/job/77332886/',
  'https://www.1111.com.tw/job/79908415/',
  'https://www.1111.com.tw/job/79720258/'],
 ['https://www.1111.com.tw/job/79839490/',
  'https://www.1111.

In [11]:
# test get job page code
url = "https://www.1111.com.tw/job/80057790/"

url.split('/')[-2] + '.html'

'80057790.html'

In [12]:
url = "https://www.1111.com.tw/job/80197415/?agent=sticktop_71952995_80197415"

url.split('/')[-2] + '.html'

'80197415.html'

In [13]:
def get_page_code(url):
    return url.split('/')[-2] + '.html'

In [14]:
get_page_code("https://www.1111.com.tw/job/80197415/?agent=sticktop_71952995_80197415")

'80197415.html'

# get useful info from page

In [15]:
URL = "https://www.1111.com.tw/job/80197415/?agent=sticktop_71952995_80197415"
resp = requests.get(URL)

soup = BeautifulSoup(resp.text, 'lxml') # Beautifulsoup Object
title_info = str(soup.find("div", {"class":"logoTitle"}))
print(title_info)

<div class="logoTitle">
<h2><span></span></h2>
<h1>網頁軟體工程師 <span>面議 / 經驗不拘 / 不拘</span></h1>
<ul class="tab">
<li class="ellipsis"><a href="/corp/71952995/" target="_blank">昇龍國際資訊科技有限公司</a></li>
<li><a href="/corp/71952995/#c4">本公司其他職缺<span id="otherJobsC"></span></a></li>
</ul>
</div>


In [16]:
def get_listpage_info(URL):
    resp = requests.get(URL)

    soup = BeautifulSoup(resp.text, 'lxml') # Beautifulsoup Object
    title_info = str(soup.find("div", {"class":"logoTitle"}))
    return title_info

## Some Challenge here

In [17]:
get_listpage_info("https://www.1111.com.tw/job/80197415/?agent=sticktop_71952995_80197415")

'<div class="logoTitle">\n<h2><span></span></h2>\n<h1>網頁軟體工程師 <span>面議 / 經驗不拘 / 不拘</span></h1>\n<ul class="tab">\n<li class="ellipsis"><a href="/corp/71952995/" target="_blank">昇龍國際資訊科技有限公司</a></li>\n<li><a href="/corp/71952995/#c4">本公司其他職缺<span id="otherJobsC"></span></a></li>\n</ul>\n</div>'

In [18]:
title_info = get_listpage_info("https://www.1111.com.tw/job/80197415/?agent=sticktop_71952995_80197415")
title = re.findall('<h1>(.+)<span>', title_info)
title

['網頁軟體工程師 ']

In [19]:
get_listpage_info("https://www.1111.com.tw/job/80057790/")

'<div class="logoTitle">\n<h2><span><a href="/corp/414424/"><img border="0" src="//images.1111.com.tw/oad/414424.jpg?2018/1/21 下午 06:41:12" style="max-width:160px;_width:160px;max-height:120px;" title="默科資訊股份有限公司"/></a></span></h2>\n<h1>Java Developer<span>面議 / 1年以上工作經驗 / 大學 以上</span></h1>\n<ul class="tab">\n<li class="ellipsis"><a href="/corp/414424/" target="_blank">默科資訊股份有限公司</a></li>\n<li><a href="/corp/414424/#c4">本公司其他職缺<span id="otherJobsC"></span></a></li>\n</ul>\n</div>'

In [20]:
title_info = get_listpage_info("https://www.1111.com.tw/job/80057790/")
title = re.findall('<h1>(.+)<span>', title_info)
title

['Java Developer']

In [21]:
def get_listpage_title(info):
    title = re.findall('<h1>(.+)<span>', info)[0]
    return title

In [22]:
company = re.findall('target=\"_blank\">(.+)</a></li>', title_info)
company[0]
# get String

'默科資訊股份有限公司'

In [23]:
def get_listpage_company(info):
    company = re.findall('target=\"_blank\">(.+)</a></li>', info)[0]
    return company

In [24]:
get_listpage_company(title_info)

'默科資訊股份有限公司'

# Combine Code

In [25]:
# test
for page in all_page:
    print(page)
    print("*****")

['https://www.1111.com.tw/job/80197415/?agent=sticktop_71952995_80197415', 'https://www.1111.com.tw/job/80057790/', 'https://www.1111.com.tw/job/79828717/', 'https://www.1111.com.tw/job/55576904/', 'https://www.1111.com.tw/job/76821519/', 'https://www.1111.com.tw/job/80231208/', 'https://www.1111.com.tw/job/76812185/', 'https://www.1111.com.tw/job/76812146/', 'https://www.1111.com.tw/job/76812082/', 'https://www.1111.com.tw/job/76812156/', 'https://www.1111.com.tw/job/80108214/', 'https://www.1111.com.tw/job/76812071/', 'https://www.1111.com.tw/job/76812055/', 'https://www.1111.com.tw/job/78573375/', 'https://www.1111.com.tw/job/79932614/', 'https://www.1111.com.tw/job/80120936/', 'https://www.1111.com.tw/job/80078480/', 'https://www.1111.com.tw/job/78356616/', 'https://www.1111.com.tw/job/77332886/', 'https://www.1111.com.tw/job/79908415/', 'https://www.1111.com.tw/job/79720258/']
*****
['https://www.1111.com.tw/job/79839490/', 'https://www.1111.com.tw/job/59308016/', 'https://www.111

In [26]:
for page in all_page:
    for _page in page:
        print(_page)

https://www.1111.com.tw/job/80197415/?agent=sticktop_71952995_80197415
https://www.1111.com.tw/job/80057790/
https://www.1111.com.tw/job/79828717/
https://www.1111.com.tw/job/55576904/
https://www.1111.com.tw/job/76821519/
https://www.1111.com.tw/job/80231208/
https://www.1111.com.tw/job/76812185/
https://www.1111.com.tw/job/76812146/
https://www.1111.com.tw/job/76812082/
https://www.1111.com.tw/job/76812156/
https://www.1111.com.tw/job/80108214/
https://www.1111.com.tw/job/76812071/
https://www.1111.com.tw/job/76812055/
https://www.1111.com.tw/job/78573375/
https://www.1111.com.tw/job/79932614/
https://www.1111.com.tw/job/80120936/
https://www.1111.com.tw/job/80078480/
https://www.1111.com.tw/job/78356616/
https://www.1111.com.tw/job/77332886/
https://www.1111.com.tw/job/79908415/
https://www.1111.com.tw/job/79720258/
https://www.1111.com.tw/job/79839490/
https://www.1111.com.tw/job/59308016/
https://www.1111.com.tw/job/79958667/
https://www.1111.com.tw/job/80260768/
https://www.1111.

In [27]:
get_listpage_info("https://www.1111.com.tw/job/80197415/?agent=sticktop_71952995_80197415")

'<div class="logoTitle">\n<h2><span></span></h2>\n<h1>網頁軟體工程師 <span>面議 / 經驗不拘 / 不拘</span></h1>\n<ul class="tab">\n<li class="ellipsis"><a href="/corp/71952995/" target="_blank">昇龍國際資訊科技有限公司</a></li>\n<li><a href="/corp/71952995/#c4">本公司其他職缺<span id="otherJobsC"></span></a></li>\n</ul>\n</div>'

In [28]:
webs = []
for page in all_page:
    for _page in page:
        webs.append(_page)
webs[:5]

['https://www.1111.com.tw/job/80197415/?agent=sticktop_71952995_80197415',
 'https://www.1111.com.tw/job/80057790/',
 'https://www.1111.com.tw/job/79828717/',
 'https://www.1111.com.tw/job/55576904/',
 'https://www.1111.com.tw/job/76821519/']

In [52]:
len(webs)

981

In [29]:
# Note: speed is very low, you can write file to speed up.
#       load and write file one by one
infos = []
for web in webs:
    infos.append(get_listpage_info(web))
infos[:3]

['<div class="logoTitle">\n<h2><span></span></h2>\n<h1>網頁軟體工程師 <span>面議 / 經驗不拘 / 不拘</span></h1>\n<ul class="tab">\n<li class="ellipsis"><a href="/corp/71952995/" target="_blank">昇龍國際資訊科技有限公司</a></li>\n<li><a href="/corp/71952995/#c4">本公司其他職缺<span id="otherJobsC"></span></a></li>\n</ul>\n</div>',
 '<div class="logoTitle">\n<h2><span><a href="/corp/414424/"><img border="0" src="//images.1111.com.tw/oad/414424.jpg?2018/1/21 下午 06:41:50" style="max-width:160px;_width:160px;max-height:120px;" title="默科資訊股份有限公司"/></a></span></h2>\n<h1>Java Developer<span>面議 / 1年以上工作經驗 / 大學 以上</span></h1>\n<ul class="tab">\n<li class="ellipsis"><a href="/corp/414424/" target="_blank">默科資訊股份有限公司</a></li>\n<li><a href="/corp/414424/#c4">本公司其他職缺<span id="otherJobsC"></span></a></li>\n</ul>\n</div>',
 '<div class="logoTitle">\n<h2><span><a href="/corp/69490798/"><img border="0" src="//images.1111.com.tw/oad/69490798.jpg?2018/1/21 下午 06:41:50" style="max-width:160px;_width:160px;max-height:120px;" title="承石資訊股份有限公

### Failed once
#### Because useful info didn't write correct

In [30]:
print("infos: " + str(type(infos)))
print("infos[0]: " + str(type(infos[0])))

infos: <class 'list'>
infos[0]: <class 'str'>


In [31]:
print(get_listpage_title(infos[0]))
print(get_listpage_company(infos[0]))
print(get_listpage_title(infos[1]))
print(get_listpage_company(infos[1]))

網頁軟體工程師 
昇龍國際資訊科技有限公司
Java Developer
默科資訊股份有限公司


In [32]:
for info in infos:
    print(info)
    print("******")

<div class="logoTitle">
<h2><span></span></h2>
<h1>網頁軟體工程師 <span>面議 / 經驗不拘 / 不拘</span></h1>
<ul class="tab">
<li class="ellipsis"><a href="/corp/71952995/" target="_blank">昇龍國際資訊科技有限公司</a></li>
<li><a href="/corp/71952995/#c4">本公司其他職缺<span id="otherJobsC"></span></a></li>
</ul>
</div>
******
<div class="logoTitle">
<h2><span><a href="/corp/414424/"><img border="0" src="//images.1111.com.tw/oad/414424.jpg?2018/1/21 下午 06:41:50" style="max-width:160px;_width:160px;max-height:120px;" title="默科資訊股份有限公司"/></a></span></h2>
<h1>Java Developer<span>面議 / 1年以上工作經驗 / 大學 以上</span></h1>
<ul class="tab">
<li class="ellipsis"><a href="/corp/414424/" target="_blank">默科資訊股份有限公司</a></li>
<li><a href="/corp/414424/#c4">本公司其他職缺<span id="otherJobsC"></span></a></li>
</ul>
</div>
******
<div class="logoTitle">
<h2><span><a href="/corp/69490798/"><img border="0" src="//images.1111.com.tw/oad/69490798.jpg?2018/1/21 下午 06:41:50" style="max-width:160px;_width:160px;max-height:120px;" title="承石資訊股份有限公司"/></a></s

</div>
******
<div class="logoTitle">
<h2><span><a href="/corp/543290/"><img border="0" src="//images.1111.com.tw/oad/543290.jpg?2018/1/21 下午 06:44:03" style="max-width:160px;_width:160px;max-height:120px;" title="先進光電科技股份有限公司"/></a></span></h2>
<h1>資訊人員<span>面議 / 2年以上工作經驗 / 大學 以上畢業</span></h1>
<ul class="tab">
<li class="ellipsis"><a href="/corp/543290/" target="_blank">先進光電科技股份有限公司</a></li>
<li><a href="/corp/543290/#c4">本公司其他職缺<span id="otherJobsC"></span></a></li>
</ul>
</div>
******
<div class="logoTitle">
<h2><span><a href="/corp/8325814/"><img border="0" src="//images.1111.com.tw/oad/8325814.jpg?2018/1/21 下午 06:44:03" style="max-width:160px;_width:160px;max-height:120px;" title="春雨工廠股份有限公司"/></a></span></h2>
<h1>資訊主管<span>面議 / 經驗不拘 / 大學 以上</span></h1>
<ul class="tab">
<li class="ellipsis"><a href="/corp/8325814/" target="_blank">春雨工廠股份有限公司</a></li>
<li><a href="/corp/8325814/#c4">本公司其他職缺<span id="otherJobsC"></span></a></li>
</ul>
</div>
******
<div class="logoTitle">
<h2><span>

In [36]:
# insert datas and filter spaces
titles = []
companys = []
for info in infos:
    titles.append(get_listpage_title(info).strip())
    companys.append(get_listpage_company(info).strip())

In [37]:
print(titles[:5])
print(companys[:5])

['網頁軟體工程師', 'Java Developer', 'Java Engineer', 'Java講師', 'Java Engineer ( Java程式設計 )']
['昇龍國際資訊科技有限公司', '默科資訊股份有限公司', '承石資訊股份有限公司', '社團法人新竹市職訓教育協會', '事必得科技有限公司']


# Json

In [44]:
data = {}
for k, v in zip(companys, titles):
    data[k] = v
data

{'(PChome Online)網路家庭國際資訊(股)公司': '軟體測試工程師(E0802)',
 '(仁心聯合醫療體系)漢坤國際醫藥股份有限公司': 'EF+BI',
 '1111人才派遣中心': '知名電子大廠誠徵Notes資深工程師',
 '1111獵才顧問中心': '【知名光電產業-旗下醫療部】SW-Android APP 技術經、副理/資深工程師',
 '1111高階獵頭顧問中心': '技術總監(756501)',
 'Be株式會社': '赴日軟體工程師(日文培訓)',
 'CHPT_中華精測科技股份有限公司': 'AI人工智慧應用系統開發工程師',
 'Cowinn International Inc': 'H5開發工程師',
 'GARMIN 台灣國際航電股份有限公司': '工業4.0百人招募專案!!製造工程部_Hadoop Cluster管理工程師_汐止',
 'Imperial Pacific International Holding Limited': 'CB Android 軟體工程師',
 'KPMG_安侯建業聯合會計師事務所': '【顧問部】顧問師/高級顧問師 (雲端應用系統開發)-319B',
 'NEW WEATHER FORCES GROUP INC': '软件测试工程师',
 'ONE STOP (Hong Kong) CO. LTD.': '資訊部主管',
 'PEAKIMAGE SERVICES INC.': 'Linux運維工程師',
 'QNAP_威聯通科技股份有限公司': '61240-軟韌體研發工程師/新竹(107年度研發替代役)',
 'R U O K TECHNOLOGY CO., LTD.': '★月薪12萬★Golang研發工程師',
 'SWISSPA瑞醫科技美容集團(總公司)(想容國際有限公司)': '美術設計/TERRIS大地之鑰精油',
 'ShopBack': '資深軟體工程師(年薪110萬起)',
 'Sky Optimum Technology': 'Mobile Software Engineer',
 'TOP CAREER': '●在日工作●大手企業Big Data Engineer #1859',
 'VIRON': '运维经理',
 'With日商偉麗管理顧問股份有限公司': '13

In [45]:
# not all jobs include in dictionary
len(data)

381

## Let's change dictionary key

In [48]:
keys = []
for web in webs:
    keys.append(get_page_code(web))
keys[:5]

['80197415.html',
 '80057790.html',
 '79828717.html',
 '55576904.html',
 '76821519.html']

In [50]:
data = {}
for k, v1, v2 in zip(keys ,companys, titles):
    data[k] = [v1, v2]
len(data)

980

In [56]:
# write json file
with open('/home/parker/etl/raw/1111raw.json', 'w') as outfile:
    json.dump(data, outfile)

In [57]:
# read json file
with open('/home/parker/etl/raw/1111raw.json') as infile:
    load_data = json.load(infile)

# Calculate job titles

In [62]:
counter = {}
for title in titles:
    if title in counter:
        counter[title] += 1
    else:
        counter[title] = 1

In [71]:
result = list(counter.items())
result[:5]

[('網頁軟體工程師', 2),
 ('Java Developer', 1),
 ('Java Engineer', 1),
 ('Java講師', 1),
 ('Java Engineer ( Java程式設計 )', 1)]

In [72]:
def get_count(x):
    return x[1]

In [75]:
sorted(result, key=get_count, reverse=True)[:20]

[('軟體工程師', 14),
 ('程式設計師', 9),
 ('Android 工程師', 4),
 ('Java工程師', 3),
 ('JAVA程式設計師', 3),
 ('資訊人員', 3),
 ('資訊工程師', 3),
 ('程式設計工程師', 3),
 ('軟體設計工程師', 3),
 ('網頁軟體工程師', 2),
 ('JAVA工程師', 2),
 ('Java架構師', 2),
 ('Java軟體工程師', 2),
 ('Java 軟體工程師', 2),
 ('Java 程式設計師', 2),
 ('Java系統設計師 (System Designer)', 2),
 ('Java程式設計教學講師', 2),
 ('JAVA資深工程師-專案約聘職', 2),
 ('網站平台 後/全端工程師 PHP/JAVA/Nodejs', 2),
 ('系統分析師', 2)]